In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.linear_model import LogisticRegression
from mord import OrdinalRidge, LogisticAT, LogisticIT, LogisticSE, LAD
np.random.seed(1)
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import normalize
from tqdm import tqdm_notebook

In [2]:
train = pd.read_pickle('train_set.pickle')
train['filename'] = train['filename'].apply(lambda x: x.split('/')[-1])
for i in tqdm_notebook(range(len(train['features']))):
    for n in range(len(train['features'][i])):
        train['features'][i][n] = normalize(train['features'][i][n].reshape(1,-1))[0]
train['features'] = train['features'].apply(lambda x: np.median(x, axis=0))
train['age'] = train['age'].apply(lambda x: np.median(x))
train['gender'] = train['gender'].apply(lambda x: np.median(x))
labels = pd.read_csv('train.txt', header=None, sep='  ')
train = pd.merge(train, labels, how='inner', left_on='filename', right_on=0)
train = train.rename(columns={0: 'name', 1: "target"})
train.dropna(inplace=True)
for i in range(1024):
    train[i] = [x[i] for x in train['features']]
train.drop(['name', 'features'], axis=1, inplace=True)
train['emotion'] = list(map(lambda x: x.split('_')[0], train['filename']))
train['emotion'] = train['emotion'].apply(lambda x: (0 if x == 'neg' else (1 if x == 'neu' else (2 if x == 'pos' else x))))
train.reset_index(drop=True, inplace=True)
train

,filename,age,gender,target,0,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,emotion
0,pos_577.jpg,34.345988,0.025728,2,0.011448,0.000000,0.025386,0.007740,0.010330,0.007339,...,0.068543,0.011214,0.037038,0.031357,0.034689,0.029508,0.016337,0.007689,0.002928,2
1,pos_583.jpg,35.299664,0.968556,2,0.015193,0.011396,0.018765,0.025794,0.000000,0.003319,...,0.017050,0.000000,0.000893,0.003356,0.073515,0.001644,0.002169,0.000000,0.000380,2
2,pos_579.jpg,9.644070,0.122276,2,0.002164,0.000000,0.000000,0.012701,0.000000,0.019303,...,0.013193,0.021045,0.000842,0.035113,0.000000,0.000000,0.001051,0.022104,0.015159,2
3,pos_585.jpg,30.467837,0.447095,2,0.030538,0.000000,0.003322,0.015780,0.001863,0.000920,...,0.007746,0.000000,0.007783,0.000000,0.014834,0.000000,0.000000,0.000766,0.000000,2
4,pos_589.jpg,28.561078,0.826333,3,0.014695,0.054258,0.001079,0.000000,0.022930,0.032229,...,0.013447,0.003056,0.007956,0.001444,0.000641,0.000000,0.002580,0.023858,0.008273,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8454,neg_1089.jpg,25.249389,0.676684,2,0.003601,0.019293,0.000000,0.016868,0.000000,0.002309,...,0.001921,0.000000,0.000760,0.000000,0.131246,0.002059,0.017291,0.000000,0.000000,0
8455,neg_1090.jpg,34.037214,0.851188,1,0.000560,0.010777,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.013190,0.000000,0.000000,0.004407,0.008003,0.000000,0.000821,0.000000,0
8456,neg_1760.jpg,29.390183,0.598478,2,0.000000,0.005021,0.000000,0.013715,0.000000,0.000000,...,0.000339,0.009092,0.000890,0.019785,0.010178,0.035904,0.000000,0.003055,0.004088,0
8457,neg_1092.jpg,36.238653,0.960967,1,0.004634,0.000035,0.007005,0.004730,0.000000,0.000000,...,0.000000,0.005929,0.000000,0.000000,0.012352,0.028212,0.001281,0.003236,0.000000,0


In [3]:
val = pd.read_pickle('val_set.pickle')
val['filename'] = val['filename'].apply(lambda x: x.split('/')[-1])
for i in tqdm_notebook(range(len(val['features']))):
    for n in range(len(val['features'][i])):
        val['features'][i][n] = normalize(val['features'][i][n].reshape(1,-1))[0]
val['features'] = val['features'].apply(lambda x: np.median(x, axis=0))
val['age'] = val['age'].apply(lambda x: np.median(x))
val['gender'] = val['gender'].apply(lambda x: np.median(x))
labels = pd.read_csv('validation.txt', header=None, sep='  ')
val = pd.merge(val, labels, how='inner', left_on='filename', right_on=0)
val = val.rename(columns={0: 'name', 1: "target"})
val.dropna(inplace=True)
for i in range(1024):
    val[i] = [x[i] for x in val['features']]
val.drop(['name', 'features'], axis=1, inplace=True)
val['emotion'] = list(map(lambda x: x.split('_')[0], val['filename']))
val['emotion'] = val['emotion'].apply(lambda x: (0 if x == 'neg' else (1 if x == 'neu' else (2 if x == 'pos' else x))))
val.reset_index(drop=True, inplace=True)
val

,filename,age,gender,target,0,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,emotion
0,pos_575.jpg,32.540987,0.702918,3,0.000000,0.000000,0.020938,0.000483,0.000000,0.008043,...,0.011914,0.000000,0.000000,0.000000,0.012291,0.000000,0.003193,0.001866,0.007781,2
1,pos_571.jpg,30.499367,0.537119,3,0.022335,0.006403,0.009814,0.097811,0.005269,0.002855,...,0.002160,0.006432,0.005128,0.039137,0.069447,0.000000,0.000000,0.006266,0.005689,2
2,pos_572.jpg,34.007962,0.344310,3,0.003601,0.000051,0.033791,0.010062,0.000000,0.022419,...,0.003022,0.021070,0.004776,0.029986,0.000000,0.024548,0.000000,0.043019,0.001862,2
3,pos_574.jpg,27.556911,0.913257,3,0.007074,0.074977,0.000000,0.000000,0.028070,0.001065,...,0.006343,0.001723,0.000751,0.003626,0.001115,0.105821,0.001400,0.000000,0.044558,2
4,pos_576.jpg,34.490304,0.901202,2,0.007449,0.010968,0.000000,0.015348,0.000000,0.009879,...,0.006879,0.001722,0.016249,0.008719,0.003100,0.001721,0.000000,0.000000,0.005987,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,neg_1076.jpg,32.542496,0.884925,2,0.002865,0.003552,0.000000,0.004541,0.000000,0.000000,...,0.000819,0.000000,0.000000,0.000000,0.000000,0.026324,0.000000,0.006546,0.000000,0
3748,neg_1071.jpg,31.486500,0.742181,2,0.000272,0.037360,0.000000,0.000000,0.000000,0.012540,...,0.017448,0.000029,0.004061,0.000000,0.011066,0.011539,0.000000,0.002868,0.002777,0
3749,neg_1078.jpg,30.142811,0.899529,2,0.000485,0.018128,0.004462,0.000000,0.000000,0.016663,...,0.003465,0.000000,0.000000,0.000000,0.004836,0.026776,0.003986,0.012005,0.000000,0
3750,neg_1077.jpg,35.505142,0.876111,2,0.000000,0.007190,0.000000,0.010082,0.000000,0.015131,...,0.017950,0.008447,0.000000,0.000000,0.004422,0.009972,0.000000,0.000000,0.006926,0


In [4]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

In [5]:
cb = Pipeline(steps=[('pca', PCA(n_components=128, whiten=False, svd_solver='auto')), ('classifier', CatBoostClassifier(depth=3, n_estimators=10000, learning_rate=0.1, loss_function='MultiClass'))])
cb.fit(train.drop(['filename', 'emotion', 'target'], axis=1), train['emotion'])

0:	learn: 1.0830318	total: 127ms	remaining: 21m 10s
1:	learn: 1.0697362	total: 171ms	remaining: 14m 12s
2:	learn: 1.0580256	total: 217ms	remaining: 12m 3s
3:	learn: 1.0472582	total: 260ms	remaining: 10m 49s
4:	learn: 1.0373275	total: 284ms	remaining: 9m 26s
5:	learn: 1.0278472	total: 302ms	remaining: 8m 23s
6:	learn: 1.0206004	total: 325ms	remaining: 7m 43s
7:	learn: 1.0138907	total: 350ms	remaining: 7m 17s
8:	learn: 1.0084478	total: 367ms	remaining: 6m 47s
9:	learn: 1.0013614	total: 383ms	remaining: 6m 22s
10:	learn: 0.9955988	total: 401ms	remaining: 6m 4s
11:	learn: 0.9897019	total: 420ms	remaining: 5m 49s
12:	learn: 0.9846054	total: 440ms	remaining: 5m 37s
13:	learn: 0.9797516	total: 462ms	remaining: 5m 29s
14:	learn: 0.9749135	total: 480ms	remaining: 5m 19s
15:	learn: 0.9697884	total: 497ms	remaining: 5m 10s
16:	learn: 0.9665344	total: 513ms	remaining: 5m 1s
17:	learn: 0.9625747	total: 530ms	remaining: 4m 54s
18:	learn: 0.9591836	total: 546ms	remaining: 4m 46s
19:	learn: 0.9549144	

166:	learn: 0.7553396	total: 3.09s	remaining: 3m 1s
167:	learn: 0.7545423	total: 3.1s	remaining: 3m 1s
168:	learn: 0.7537149	total: 3.12s	remaining: 3m 1s
169:	learn: 0.7528887	total: 3.13s	remaining: 3m
170:	learn: 0.7522561	total: 3.14s	remaining: 3m
171:	learn: 0.7515160	total: 3.15s	remaining: 3m
172:	learn: 0.7509755	total: 3.17s	remaining: 2m 59s
173:	learn: 0.7503868	total: 3.18s	remaining: 2m 59s
174:	learn: 0.7497443	total: 3.19s	remaining: 2m 59s
175:	learn: 0.7491515	total: 3.21s	remaining: 2m 58s
176:	learn: 0.7487488	total: 3.22s	remaining: 2m 58s
177:	learn: 0.7480988	total: 3.24s	remaining: 2m 58s
178:	learn: 0.7473831	total: 3.25s	remaining: 2m 58s
179:	learn: 0.7467258	total: 3.27s	remaining: 2m 58s
180:	learn: 0.7460986	total: 3.28s	remaining: 2m 58s
181:	learn: 0.7452897	total: 3.3s	remaining: 2m 57s
182:	learn: 0.7447130	total: 3.31s	remaining: 2m 57s
183:	learn: 0.7440621	total: 3.32s	remaining: 2m 57s
184:	learn: 0.7434293	total: 3.34s	remaining: 2m 57s
185:	learn

322:	learn: 0.6688692	total: 5.69s	remaining: 2m 50s
323:	learn: 0.6684522	total: 5.7s	remaining: 2m 50s
324:	learn: 0.6681732	total: 5.72s	remaining: 2m 50s
325:	learn: 0.6677258	total: 5.74s	remaining: 2m 50s
326:	learn: 0.6672818	total: 5.76s	remaining: 2m 50s
327:	learn: 0.6667321	total: 5.77s	remaining: 2m 50s
328:	learn: 0.6664502	total: 5.79s	remaining: 2m 50s
329:	learn: 0.6658345	total: 5.81s	remaining: 2m 50s
330:	learn: 0.6653955	total: 5.82s	remaining: 2m 50s
331:	learn: 0.6650092	total: 5.84s	remaining: 2m 49s
332:	learn: 0.6645429	total: 5.86s	remaining: 2m 50s
333:	learn: 0.6640073	total: 5.88s	remaining: 2m 50s
334:	learn: 0.6635669	total: 5.9s	remaining: 2m 50s
335:	learn: 0.6631123	total: 5.92s	remaining: 2m 50s
336:	learn: 0.6626627	total: 5.94s	remaining: 2m 50s
337:	learn: 0.6621856	total: 5.96s	remaining: 2m 50s
338:	learn: 0.6617538	total: 5.98s	remaining: 2m 50s
339:	learn: 0.6614553	total: 6s	remaining: 2m 50s
340:	learn: 0.6610165	total: 6.01s	remaining: 2m 50

484:	learn: 0.6098324	total: 8.71s	remaining: 2m 50s
485:	learn: 0.6094791	total: 8.73s	remaining: 2m 50s
486:	learn: 0.6090389	total: 8.75s	remaining: 2m 50s
487:	learn: 0.6087770	total: 8.77s	remaining: 2m 50s
488:	learn: 0.6085599	total: 8.78s	remaining: 2m 50s
489:	learn: 0.6082627	total: 8.79s	remaining: 2m 50s
490:	learn: 0.6079599	total: 8.81s	remaining: 2m 50s
491:	learn: 0.6075534	total: 8.82s	remaining: 2m 50s
492:	learn: 0.6071922	total: 8.85s	remaining: 2m 50s
493:	learn: 0.6068560	total: 8.86s	remaining: 2m 50s
494:	learn: 0.6064744	total: 8.88s	remaining: 2m 50s
495:	learn: 0.6062562	total: 8.9s	remaining: 2m 50s
496:	learn: 0.6061250	total: 8.91s	remaining: 2m 50s
497:	learn: 0.6057431	total: 8.93s	remaining: 2m 50s
498:	learn: 0.6053756	total: 8.95s	remaining: 2m 50s
499:	learn: 0.6050909	total: 8.97s	remaining: 2m 50s
500:	learn: 0.6046784	total: 8.98s	remaining: 2m 50s
501:	learn: 0.6042248	total: 9s	remaining: 2m 50s
502:	learn: 0.6039296	total: 9.02s	remaining: 2m 5

640:	learn: 0.5643175	total: 11.6s	remaining: 2m 49s
641:	learn: 0.5640231	total: 11.7s	remaining: 2m 49s
642:	learn: 0.5637849	total: 11.7s	remaining: 2m 49s
643:	learn: 0.5636283	total: 11.7s	remaining: 2m 49s
644:	learn: 0.5633857	total: 11.7s	remaining: 2m 50s
645:	learn: 0.5631934	total: 11.8s	remaining: 2m 50s
646:	learn: 0.5630419	total: 11.8s	remaining: 2m 50s
647:	learn: 0.5628105	total: 11.8s	remaining: 2m 50s
648:	learn: 0.5624655	total: 11.8s	remaining: 2m 50s
649:	learn: 0.5621305	total: 11.8s	remaining: 2m 50s
650:	learn: 0.5619607	total: 11.8s	remaining: 2m 50s
651:	learn: 0.5617607	total: 11.9s	remaining: 2m 50s
652:	learn: 0.5615605	total: 11.9s	remaining: 2m 50s
653:	learn: 0.5613315	total: 11.9s	remaining: 2m 50s
654:	learn: 0.5610389	total: 11.9s	remaining: 2m 50s
655:	learn: 0.5606249	total: 11.9s	remaining: 2m 50s
656:	learn: 0.5603697	total: 12s	remaining: 2m 50s
657:	learn: 0.5601366	total: 12s	remaining: 2m 50s
658:	learn: 0.5599351	total: 12s	remaining: 2m 50s

...

9470:	learn: 0.0533105	total: 2m 56s	remaining: 9.85s
9471:	learn: 0.0533056	total: 2m 56s	remaining: 9.83s
9472:	learn: 0.0533010	total: 2m 56s	remaining: 9.81s
9473:	learn: 0.0532929	total: 2m 56s	remaining: 9.79s
9474:	learn: 0.0532921	total: 2m 56s	remaining: 9.77s
9475:	learn: 0.0532905	total: 2m 56s	remaining: 9.75s
9476:	learn: 0.0532829	total: 2m 56s	remaining: 9.74s
9477:	learn: 0.0532619	total: 2m 56s	remaining: 9.72s
9478:	learn: 0.0532462	total: 2m 56s	remaining: 9.7s
9479:	learn: 0.0532371	total: 2m 56s	remaining: 9.68s
9480:	learn: 0.0532283	total: 2m 56s	remaining: 9.66s
9481:	learn: 0.0532278	total: 2m 56s	remaining: 9.64s
9482:	learn: 0.0532226	total: 2m 56s	remaining: 9.62s
9483:	learn: 0.0532142	total: 2m 56s	remaining: 9.61s
9484:	learn: 0.0532140	total: 2m 56s	remaining: 9.59s
9485:	learn: 0.0532132	total: 2m 56s	remaining: 9.57s
9486:	learn: 0.0532054	total: 2m 56s	remaining: 9.55s
9487:	learn: 0.0532006	total: 2m 56s	remaining: 9.53s
9488:	learn: 0.0531829	total:

9624:	learn: 0.0521500	total: 2m 59s	remaining: 6.98s
9625:	learn: 0.0521470	total: 2m 59s	remaining: 6.96s
9626:	learn: 0.0521410	total: 2m 59s	remaining: 6.94s
9627:	learn: 0.0521404	total: 2m 59s	remaining: 6.92s
9628:	learn: 0.0521335	total: 2m 59s	remaining: 6.91s
9629:	learn: 0.0521286	total: 2m 59s	remaining: 6.89s
9630:	learn: 0.0521282	total: 2m 59s	remaining: 6.87s
9631:	learn: 0.0521201	total: 2m 59s	remaining: 6.85s
9632:	learn: 0.0521034	total: 2m 59s	remaining: 6.83s
9633:	learn: 0.0520969	total: 2m 59s	remaining: 6.81s
9634:	learn: 0.0520957	total: 2m 59s	remaining: 6.79s
9635:	learn: 0.0520952	total: 2m 59s	remaining: 6.77s
9636:	learn: 0.0520933	total: 2m 59s	remaining: 6.75s
9637:	learn: 0.0520868	total: 2m 59s	remaining: 6.74s
9638:	learn: 0.0520823	total: 2m 59s	remaining: 6.72s
9639:	learn: 0.0520733	total: 2m 59s	remaining: 6.7s
9640:	learn: 0.0520607	total: 2m 59s	remaining: 6.68s
9641:	learn: 0.0520554	total: 2m 59s	remaining: 6.66s
9642:	learn: 0.0520526	total:

9789:	learn: 0.0509936	total: 3m 2s	remaining: 3.9s
9790:	learn: 0.0509899	total: 3m 2s	remaining: 3.89s
9791:	learn: 0.0509829	total: 3m 2s	remaining: 3.87s
9792:	learn: 0.0509771	total: 3m 2s	remaining: 3.85s
9793:	learn: 0.0509643	total: 3m 2s	remaining: 3.83s
9794:	learn: 0.0509498	total: 3m 2s	remaining: 3.81s
9795:	learn: 0.0509426	total: 3m 2s	remaining: 3.79s
9796:	learn: 0.0509330	total: 3m 2s	remaining: 3.77s
9797:	learn: 0.0509279	total: 3m 2s	remaining: 3.76s
9798:	learn: 0.0509090	total: 3m 2s	remaining: 3.74s
9799:	learn: 0.0508891	total: 3m 2s	remaining: 3.72s
9800:	learn: 0.0508860	total: 3m 2s	remaining: 3.7s
9801:	learn: 0.0508709	total: 3m 2s	remaining: 3.68s
9802:	learn: 0.0508689	total: 3m 2s	remaining: 3.66s
9803:	learn: 0.0508583	total: 3m 2s	remaining: 3.65s
9804:	learn: 0.0508528	total: 3m 2s	remaining: 3.63s
9805:	learn: 0.0508368	total: 3m 2s	remaining: 3.61s
9806:	learn: 0.0508291	total: 3m 2s	remaining: 3.59s
9807:	learn: 0.0508131	total: 3m 2s	remaining: 3

9947:	learn: 0.0498243	total: 3m 5s	remaining: 967ms
9948:	learn: 0.0498058	total: 3m 5s	remaining: 949ms
9949:	learn: 0.0498029	total: 3m 5s	remaining: 930ms
9950:	learn: 0.0497926	total: 3m 5s	remaining: 912ms
9951:	learn: 0.0497916	total: 3m 5s	remaining: 893ms
9952:	learn: 0.0497895	total: 3m 5s	remaining: 874ms
9953:	learn: 0.0497854	total: 3m 5s	remaining: 856ms
9954:	learn: 0.0497814	total: 3m 5s	remaining: 837ms
9955:	learn: 0.0497666	total: 3m 5s	remaining: 818ms
9956:	learn: 0.0497629	total: 3m 5s	remaining: 800ms
9957:	learn: 0.0497540	total: 3m 5s	remaining: 781ms
9958:	learn: 0.0497386	total: 3m 5s	remaining: 763ms
9959:	learn: 0.0497271	total: 3m 5s	remaining: 744ms
9960:	learn: 0.0497226	total: 3m 5s	remaining: 726ms
9961:	learn: 0.0497132	total: 3m 5s	remaining: 707ms
9962:	learn: 0.0496888	total: 3m 5s	remaining: 688ms
9963:	learn: 0.0496864	total: 3m 5s	remaining: 670ms
9964:	learn: 0.0496724	total: 3m 5s	remaining: 651ms
9965:	learn: 0.0496563	total: 3m 5s	remaining:

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=128,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x0000020251D24288>)],
         verbose=False)

In [6]:
from sklearn.metrics import classification_report
print('TRAIN:\n',classification_report(train['emotion'], cb.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION:\n',classification_report(val['emotion'], cb.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))
print('TRAIN MSE: ', mean_squared_error(train['emotion'], cb.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION MSE: ', mean_squared_error(val['emotion'], cb.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))

TRAIN:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2226
           1       1.00      1.00      1.00      2635
           2       1.00      1.00      1.00      3598

    accuracy                           1.00      8459
   macro avg       1.00      1.00      1.00      8459
weighted avg       1.00      1.00      1.00      8459

VALIDATION:
               precision    recall  f1-score   support

           0       0.66      0.58      0.62      1060
           1       0.60      0.62      0.61      1128
           2       0.79      0.84      0.81      1564

    accuracy                           0.70      3752
   macro avg       0.68      0.68      0.68      3752
weighted avg       0.70      0.70      0.70      3752

TRAIN MSE:  0.001536824683768767
VALIDATION MSE:  0.48800639658848616


In [35]:
from sklearn.svm import SVC
svm = Pipeline(steps=[('pca', PCA(n_components=256, whiten=True)),('classifier', SVC(C=15, kernel='rbf'))])
svm.fit(train.drop(['filename', 'emotion', 'target'], axis=1), train['emotion'])

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=256,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=True)),
                ('classifier',
                 SVC(C=15, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [36]:
print('TRAIN:\n',classification_report(train['emotion'], svm.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION:\n',classification_report(val['emotion'], svm.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))
# print('TRAIN MSE: ', mean_squared_error(train['emotion'], svm.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
# print('VALIDATION MSE: ', mean_squared_error(val['emotion'], svm.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))

TRAIN:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2226
           1       1.00      1.00      1.00      2635
           2       1.00      1.00      1.00      3598

    accuracy                           1.00      8459
   macro avg       1.00      1.00      1.00      8459
weighted avg       1.00      1.00      1.00      8459

VALIDATION:
               precision    recall  f1-score   support

           0       0.65      0.56      0.60      1060
           1       0.59      0.63      0.61      1128
           2       0.80      0.84      0.82      1564

    accuracy                           0.70      3752
   macro avg       0.68      0.67      0.68      3752
weighted avg       0.69      0.70      0.69      3752



In [7]:
from sklearn.linear_model import LogisticRegression
logreg = Pipeline(steps=[('pca', PCA(n_components=16, whiten=True)), ('classifier', LogisticRegression(C=7, solver='newton-cg'))])
logreg.fit(train.drop(['filename', 'emotion', 'target'], axis=1), train['emotion'])

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=16,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=True)),
                ('classifier',
                 LogisticRegression(C=7, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='newton-cg', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [8]:
print('TRAIN:\n',classification_report(train['emotion'], logreg.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION:\n',classification_report(val['emotion'], logreg.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))
print('TRAIN MSE: ', mean_squared_error(train['emotion'], logreg.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION MSE: ', mean_squared_error(val['emotion'], logreg.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))

TRAIN:
               precision    recall  f1-score   support

           0       0.52      0.48      0.50      2226
           1       0.52      0.46      0.49      2635
           2       0.72      0.81      0.76      3598

    accuracy                           0.61      8459
   macro avg       0.58      0.58      0.58      8459
weighted avg       0.60      0.61      0.61      8459

VALIDATION:
               precision    recall  f1-score   support

           0       0.59      0.49      0.53      1060
           1       0.54      0.52      0.53      1128
           2       0.70      0.80      0.75      1564

    accuracy                           0.63      3752
   macro avg       0.61      0.60      0.60      3752
weighted avg       0.62      0.63      0.62      3752

TRAIN MSE:  0.643456673365646
VALIDATION MSE:  0.654317697228145


In [9]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
params = {'classifier__C':range(0,31),
         'classifier__max_iter':[100, 200, 500, 1000],
          'classifier__penalty':['l2', 'l1'],
          'classifier__solver':['newton-cg', 'sag'],
         'pca__n_components':[16,32,64,128,256],
         'pca__whiten':[True, False]}
fold = StratifiedKFold(n_splits=5, shuffle=True)
grid = GridSearchCV(logreg, param_grid=params, n_jobs=-1, cv=fold, scoring='accuracy', verbose=1)
grid.fit(train.drop(['filename', 'emotion', 'target'], axis=1), train['emotion'])

Fitting 5 folds for each of 4960 candidates, totalling 24800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 44.8min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 54.0min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 66.4min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 77.6min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 91.6min
[Parallel(n_jobs=-1)]: Done 11234 tasks      |

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=16, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=True)),
                                       ('classifier',
                                        LogisticRegression(C=7,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1...
                                verbose=False),
             iid='deprecated', n_jo

In [10]:
grid.best_params_

{'classifier__C': 12,
 'classifier__max_iter': 100,
 'classifier__penalty': 'l2',
 'classifier__solver': 'newton-cg',
 'pca__n_components': 256,
 'pca__whiten': False}

In [12]:
logreg = Pipeline(steps=[('pca', PCA(n_components=256, whiten=False)), ('classifier', LogisticRegression(C=12, solver='newton-cg', max_iter=100))])
logreg.fit(train.drop(['filename', 'emotion', 'target'], axis=1), train['emotion'])
print('TRAIN:\n',classification_report(train['emotion'], logreg.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION:\n',classification_report(val['emotion'], logreg.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))
print('TRAIN MSE: ', mean_squared_error(train['emotion'], logreg.predict(train.drop(['filename', 'target', 'emotion'], axis=1))))
print('VALIDATION MSE: ', mean_squared_error(val['emotion'], logreg.predict(val.drop(['filename', 'target', 'emotion'], axis=1))))

TRAIN:
               precision    recall  f1-score   support

           0       0.65      0.62      0.63      2226
           1       0.64      0.65      0.64      2635
           2       0.84      0.85      0.84      3598

    accuracy                           0.73      8459
   macro avg       0.71      0.71      0.71      8459
weighted avg       0.73      0.73      0.73      8459

VALIDATION:
               precision    recall  f1-score   support

           0       0.63      0.58      0.61      1060
           1       0.61      0.61      0.61      1128
           2       0.81      0.85      0.82      1564

    accuracy                           0.70      3752
   macro avg       0.68      0.68      0.68      3752
weighted avg       0.70      0.70      0.70      3752

TRAIN MSE:  0.421208180636009
VALIDATION MSE:  0.4968017057569296


In [44]:
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Activation
from tensorflow.keras import Model, Sequential
model = Sequential()   
model.add(Conv1D(128, 3, padding='same', input_shape=(1026, 1)))
model.add(MaxPooling1D(3))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(3))
model.add(Activation('softmax'))

In [45]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [46]:
train_scaled_conv = np.expand_dims(train.drop(['filename', 'emotion', 'target'], axis=1), axis=2)
val_scaled_conv = np.expand_dims(val.drop(['filename', 'emotion', 'target'], axis=1), axis=2)

In [47]:
model.fit(train_scaled_conv, target, epochs=100, batch_size=32, validation_split=0.1)

Train on 7613 samples, validate on 846 samples
Epoch 1/100
7613/7613 [==============================] - 15s 2ms/sample - loss: 1.0264 - acc: 0.4822 - val_loss: 0.8122 - val_acc: 0.6868
Epoch 2/100
7613/7613 [==============================] - 15s 2ms/sample - loss: 0.9756 - acc: 0.5089 - val_loss: 0.8742 - val_acc: 0.6241
Epoch 3/100
7613/7613 [==============================] - 15s 2ms/sample - loss: 0.9315 - acc: 0.5463 - val_loss: 0.9258 - val_acc: 0.5437
Epoch 4/100
7613/7613 [==============================] - 16s 2ms/sample - loss: 0.8784 - acc: 0.5784 - val_loss: 0.8345 - val_acc: 0.6371
Epoch 5/100
7613/7613 [==============================] - 15s 2ms/sample - loss: 0.8295 - acc: 0.6161 - val_loss: 0.8074 - val_acc: 0.6537
Epoch 6/100
7613/7613 [==============================] - 16s 2ms/sample - loss: 0.7853 - acc: 0.6427 - val_loss: 0.9139 - val_acc: 0.5496
Epoch 7/100
7613/7613 [==============================] - 16s 2ms/sample - loss: 0.7426 - acc: 0.6692 - val_loss: 0.8549 - val

In [42]:
train_target = train['target'].apply(lambda x: (0 if x == 1 else (1 if x == 2 else (2 if x == 3 else x))))
val_target = val['target'].apply(lambda x: (0 if x == 1 else (1 if x == 2 else (2 if x == 3 else x))))

In [48]:
print('TRAIN:\n',classification_report(train_target, model.predict_classes(train_scaled_conv)))
print('VALIDATION:\n',classification_report(val_target, model.predict_classes(val_scaled_conv)))
print('TRAIN MSE: ', mean_squared_error(train_target, model.predict_classes(train_scaled_conv)))
print('VALIDATION MSE: ', mean_squared_error(val_target, model.predict_classes(val_scaled_conv)))

TRAIN:
               precision    recall  f1-score   support

           0       0.87      0.96      0.91      2439
           1       0.97      0.91      0.94      4239
           2       0.96      0.96      0.96      1781

    accuracy                           0.94      8459
   macro avg       0.93      0.94      0.94      8459
weighted avg       0.94      0.94      0.94      8459

VALIDATION:
               precision    recall  f1-score   support

           0       0.43      0.48      0.45      1315
           1       0.36      0.45      0.40      1275
           2       0.47      0.29      0.36      1162

    accuracy                           0.41      3752
   macro avg       0.42      0.41      0.40      3752
weighted avg       0.42      0.41      0.41      3752

TRAIN MSE:  0.08476179217401585
VALIDATION MSE:  0.9818763326226013
